In [1]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.applications import InceptionResNetV2
from tensorflow.keras import models, layers, optimizers, regularizers
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator 
from keras.constraints import maxnorm
import numpy as np

In [ ]:
# Load Data
def load_data():
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train = x_train.astype('float32') 
    x_test = x_test.astype('float32') 

    #z-score
    mean = np.mean(x_train,axis=(0,1,2,3))
    std = np.std(x_train,axis=(0,1,2,3))
    x_train = (x_train-mean)/(std+1e-7)
    x_test = (x_test-mean)/(std+1e-7)
    
    y_train = np_utils.to_categorical(y_train, 10)

In [ ]:
def load_cnn():
    conv_base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    conv_base.summary()
    return conv_base

In [ ]:
def train(x_train, y_train, x_test, y_test, conv_base):
    model = models.Sequential()
    model.add(layers.UpSampling2D(7, 7))
    model.add(conv_base)

    model.add(layers.Flatten())
    model.add(Dense(4096, activation='elu', kernel_constraint=maxnorm(3))) 
    model.add(BatchNormalization())
    model.add(Dropout(0.5)) 

    model.add(Dense(4096, activation='elu', kernel_constraint=maxnorm(3))) 
    model.add(BatchNormalization())
    model.add(Dropout(0.5)) 

    model.add(Dense(1000, activation='elu', kernel_constraint=maxnorm(3))) 
    model.add(BatchNormalization())
    model.add(Dropout(0.5)) 

    model.add(layers.Dense(10, activation='softmax'))
    return model

In [ ]:
def data_augmentation(x_train):
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        vertical_flip=False
    )
    datagen.fit(x_train)
    return datagen